In [9]:
# easy_install pyscreenshot

In [1]:
import pyscreenshot
import pyautogui, sys
import operator
import math
import numpy as np
from PIL import Image
import time
from pynput import mouse
from pynput.mouse import Listener, Button, Controller

p = (295, 179)
cell_width = 16
cell_hight = 16.0
cell_dimentions = (30,16)
cell_margine = 3 #I use this margine get into the core cell
cell_upper_margine = 1
cell_margine_2 = 6 #I use this margine get into the core cell
cell_upper_margine_2 = 2

pyautogui.FAILSAFE = False
pyautogui.PAUSE = 0

In [2]:
def cSub(p1, p2):
    r,g,b,a = p1
    x,y,z,s = p2
    d1 = (abs(r-x) )
    d2 = (abs(g-y))
    d3 = (abs(b-z))
    #d4 = ((a-s) ** 2)
    '''if(d1 < 10):
        d1 = 0
    if(d2 < 10):
        d2 = 0
    if(d3 < 10):
        d3 = 0
    #if(d4 < 100):
     #   d4 = 0'''
    return (d1+d2+d3)

def calculateDiff(a,b):
    return sum(list(map(cSub, a, b)))
def findBestMatch(a):
    diff = []
    #diff.append(calculateDiff(a, i4))
    for cell in cell_list:
        cost = calculateDiff(a, cell)
        diff.append(cost)
        if(cost < 15): break
    #print((diff))
    return(diff.index(min(diff)))
def getCellImg(x,y):
    'this method is very slow takes about .2 second per cell, which does not make sense to work with it'
    return pyscreenshot.grab(bbox=(p[0] + cell_upper_margine + (x*cell_width),
                                   p[1] + cell_upper_margine + (y*cell_width),
                                   p[0] - cell_margine + ((x+1)*cell_width),
                                   p[1] - cell_margine + ((y+1)*cell_hight) ))

def getCellImgFromImage(img,x,y):
    'this method is much better performance .02 seconds for each cell'
    return img.crop((cell_upper_margine_2 + (x*cell_width*2),
                                   cell_upper_margine_2 + (y*cell_width*2),
                                   ((x+1)*cell_width*2) - cell_margine_2,
                                   ((y+1)*cell_hight*2) - cell_margine_2 ))

def getFullImg():
    return pyscreenshot.grab(bbox=(p[0] ,
                                   p[1] ,
                                   p[0] + (cell_width * cell_dimentions[0]),
                                   p[1] + (cell_hight * cell_dimentions[1]) ))
def getFullImgWithCoordinations(x,y):
    return pyscreenshot.grab(bbox=(x ,
                                   y ,
                                   x + (cell_width * cell_dimentions[0]),
                                   y + (cell_hight * cell_dimentions[1]) ))
def moveMouseToCell(x,y, cell_x, cell_y):
    pyautogui.moveTo(x+(cell_x*cell_width) + cell_upper_margine_2 + 3, y+(cell_y*cell_width)+cell_margine+2)
def mouseAction(action, x, y, cell_x, cell_y):
    if(action == 'r'):
        pyautogui.mouseDown(x=None, y=None, button='right')
        pyautogui.mouseUp(x=None, y=None, button='right')
    elif(action == 'd'):
        pyautogui.mouseDown(x=None, y=None, button='left')
        pyautogui.mouseDown(x=None, y=None, button='right')
        pyautogui.mouseUp(x=None, y=None, button='left')
        pyautogui.mouseUp(x=None, y=None, button='right')

In [3]:
def initiateCells():
    getCellImg(0,0).save('cell_one.png')
    getCellImg(0,1).save('cell_two.png')
    getCellImg(1,1).save('cell_three.png')
    getCellImg(9,1).save('cell_four.png')
    getCellImg(24,7).save('cell_five.png')
    getCellImg(3,4).save('cell_empty.png')
    getCellImg(16,1).save('cell_covered.png')
    getCellImg(1,0).save('cell_marked.png')
    getCellImg(16,0).save('cell_blow.png')
    getCellImg(9,0).save('cell_bad.png')
    getCellImg(10,0).save('cell_bad_1.png')
    return None
def initiateCellsWithImage():
    im = Image.open('training.png')
    #im.show()
    getCellImgFromImage(im,0,0).save('tf/cell_one.png')
    getCellImgFromImage(im,0,1).save('tf/cell_two.png')
    getCellImgFromImage(im,1,1).save('tf/cell_three.png')
    getCellImgFromImage(im,9,1).save('tf/cell_four.png')
    getCellImgFromImage(im,24,7).save('tf/cell_five.png')
    getCellImgFromImage(im,3,4).save('tf/cell_empty.png')
    getCellImgFromImage(im,16,1).save('tf/cell_covered.png')
    getCellImgFromImage(im,1,0).save('tf/cell_marked.png')
    getCellImgFromImage(im,16,0).save('tf/cell_blow.png')
    getCellImgFromImage(im,9,0).save('tf/cell_bad.png')
    getCellImgFromImage(im,10,0).save('tf/cell_bad_1.png')
    im = Image.open('training_1.png')
    getCellImgFromImage(im,9,6).save('tf/cell_six.png')    
    return None
#initiateCellsWithImage()
#initiateCells() #it is danger to use this function

In [4]:
i1 = list(Image.open("cell_one.png").getdata())
i2 = list(Image.open("cell_two.png").getdata())
i3 = list(Image.open("cell_three.png").getdata())
i4 = list(Image.open("cell_four.png").getdata())
i5 = list(Image.open("cell_five.png").getdata())
i6 = list(Image.open("cell_six.png").getdata())
e_cell = list(Image.open("cell_empty.png").getdata())
c_cell = list(Image.open("cell_covered.png").getdata())
m_cell = list(Image.open("cell_marked.png").getdata())
blow_cell = list(Image.open("cell_blow.png").getdata())
bad_cell = list(Image.open("cell_bad.png").getdata())
bad_1_cell = list(Image.open("cell_bad_1.png").getdata())
cell_list = [i1,i2,i3,i4,i5,i6,e_cell,c_cell,
             m_cell, blow_cell, bad_cell, bad_1_cell]
cell_names = {0:'one', 1:'two', 2:'three', 3:'four', 4:'five', 5:'six', 6:'empty'
              , 7:'covered', 8:'marked', 9:'blow', 10:'bad', 11:'bad_1'}
cell_numbers = {'one' : 0, 'two':1, 'three':2, 'four':3, 'five':4, 'six':5, 'empty':6
              , 'covered':7, 'marked':8, 'blow':9, 'bad':10, 'bad_1':11}

In [6]:

def on_click(x, y, button, pressed):
    #print('{0} at {1}'.format(
        #'Pressed' if pressed else 'Released',
        #(x, y)))
    if not pressed:
        play(x,y)
        # Stop listener
        return False
    
# Collect events until released
with Listener(
        on_click=on_click,
        ) as listener:
    listener.join()

NameError: name 'play' is not defined

In [7]:
def manipulate_cell_matrix(fileName):
    #print(fileName)
    cell_matrix = []
    im = Image.open(fileName)
    #im.show()
    for i in range(cell_dimentions[0]):
        cell_matrix.append([])
        for j in range(cell_dimentions[1]):#cell_dimentions[1]):
            ic = list(getCellImgFromImage(im,i,j).getdata())
            #cell_matrix[i].append(cell_names.get(findBestMatch(ic)))
            cell_matrix[i].append((findBestMatch(ic)))
    #print(np.transpose(cell_matrix))
    return np.transpose(cell_matrix)

In [8]:
def verify_bouble_click(matrix, x,y):
    for i in range(x-1, x+2):
        if i<0 or i+1>cell_dimentions[1]:
            continue
        for j in range(y-1, y+2):
            if j<0 or j+1>cell_dimentions[0]:
                continue
            if(matrix[i][j] == cell_numbers.get('covered')):
                return True
    return False
def get_double_click_action(matrix, x, y):
    dca = []
    for i in range(x-1, x+2):
        if i<0 or i+1>cell_dimentions[1]:
            continue
        for j in range(y-1, y+2):
            if j<0 or j+1>cell_dimentions[0]:
                continue
            if(matrix[i][j] < 6):
                dca.append(('d', (i,j)))
    return dca
def investigate_action(matrix, x, y):
    actions = []
    ni = np.zeros(9).reshape(3,3)
    covered = []
    marked = []
    for i in range(x-1, x+2):
        if i<0 or i+1>cell_dimentions[1]:
            continue
        for j in range(y-1, y+2):
            if j<0 or j+1>cell_dimentions[0]:
                continue
            if(matrix[i][j] == cell_numbers.get('covered')):
                covered.append((i,j))
            if(matrix[i][j] == cell_numbers.get('marked')):
                marked.append((i,j))
    if(len(marked) + len(covered) == matrix[x][y]+1):
        for i in covered:
            actions.append(('r', i))
    return actions
            
def get_actions(matrix):
    global_actions = []
    double_click_actions = []
    r_actions = False
    for i in range(cell_dimentions[1]):
        for j in range(cell_dimentions[0]):
            #print('position is ', i, ' -- ', j)
            if(matrix[i][j]>cell_numbers.get('marked')):
                return [], False
            if(matrix[i][j] == cell_numbers.get('marked')):
                #if(i == 12 and j == 10):
                    #print('found marked')
                    #print(get_double_click_action(matrix, i, j))
                dca = get_double_click_action(matrix, i, j)
                for a2 in dca:
                    if(a2 in double_click_actions):
                        pass
                    else:
                        double_click_actions.append(a2)
            if(matrix[i][j] < cell_numbers.get('empty')):
                actions = investigate_action(matrix, i, j)
                if(len(actions) > 0):
                    for a in actions:
                        if(a in global_actions):
                            pass
                        else:
                            global_actions.append(a)
                            dca = get_double_click_action(matrix, a[1][0], a[1][1])
                            for a2 in dca:
                                if(a2 in double_click_actions):
                                    pass
                                else:
                                    double_click_actions.append(a2)
    if(len(global_actions) > 0):
        r_actions = True
    for a in double_click_actions:
        #if(a[1][0] == 13 and a[1][1] == 10):
            #print('action d -- ',a)
            #print(matrix[a[1][0]][a[1][1]])
            #print(verify_bouble_click(matrix, a[1][0], a[1][1]))
        if(verify_bouble_click(matrix, a[1][0], a[1][1])):
            global_actions.append(a)
    #print(global_actions)
    return global_actions, r_actions

def execute_actions(x, y, actions):
    for a in actions:
        action, cord = a
        moveMouseToCell(x,y,cord[1], cord[0])
        mouseAction(action,x,y, cord[1], cord[0])

In [9]:
def play(mousex, mousey):
    count = 0
    while(True):
        t = "snapshots/" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime()) + '.png'
        getFullImgWithCoordinations(mousex, mousey).save(t)
        matrix = manipulate_cell_matrix(t)
        actions, ra = get_actions(matrix)
        if(ra):
            count = 0
        else:
            count += 1
        if(len(actions) < 1 or count > 1):
            actions = []
            s = input('continue? y/n')
            if(s == 'n'):
                break
        execute_actions(mousex, mousey, actions)

In [90]:
from pynput import mouse
from pynput.mouse import Listener

#pyautogui.moveTo(100, 200, duration=3)
print(dir(mouse.Listener))
def on_move(x, y):
    print('Pointer moved to {0}'.format(
        (x, y)))

def on_click(x, y, button, pressed):
    print('{0} at {1}'.format(
        'Pressed' if pressed else 'Released',
        (x, y)))
    if not pressed:
        # Stop listener
        return False

def on_scroll(x, y, dx, dy):
    print('Scrolled {0}'.format(
        (x, y)))

# Collect events until released
with Listener(
        #on_move=on_move,
        on_click=on_click,
        #on_scroll=on_scroll
        ) as listener:
    listener.join()

#print('starting')
#print(cell_dimentions)
#for i in range(cell_dimentions[0]):
#    cell_matrix.append([])
#    print(i)
#    for j in range(cell_dimentions[1]):
#        ic = list(getCellImg(j,i).getdata())
#        cell_matrix[i].append(cell_names.get(findBestMatch(ic)))'''
#print(dir(pyscreenshot.loader))
#print(pyscreenshot.about)
#im = Image.open('training.png')
#cell_matrix = []
#for i in range(cell_dimentions[0]):
#    cell_matrix.append([])
#    for j in range(cell_dimentions[1]):#cell_dimentions[1]):
#        ic = list(getCellImgFromImage(im,i,j).getdata())
#        #cell_matrix[i].append(cell_names.get(findBestMatch(ic)))
#        cell_matrix[i].append((findBestMatch(ic)))
#
#cell_matrix = np.transpose(cell_matrix)
#for i in range(cell_dimentions[1]):
#    print(cell_matrix[i])
#    print('------------')
#fcell = getCellImgFromImage(im, 28, 14)
#print(cell_names.get(findBestMatch(fcell.getdata())))
#imc = im.crop((0,0,32,32))
#imc.show()
#im.show()
#im.show()
#print(dir(im))
#im2 = Image.frombytes('RGBA',(960,512),im.tobytes())
#im2.show()
#np_frame = np.array(im_frame.getdata())
#print(cell_matrix)
#fi = getFullImg()
#fi.show()
#print(dir(fi))
#b = fi.getpixel
#print((b))
#b = fi.getpixel((0,0))
#print(b)
#b = fi.getpixel((511,959))
#print(b[0])
#getCellImg(29,15).show()
#ic = list(getCellImg(29,0).getdata())
#print(cell_names.get(findBestMatch(ic)))
#getCellImg(29,15).show()

# part of the screen
#sh=pyscreenshot.grab(bbox=(10,10,11,12))
#screenshot.show()
#print((sh.getpixel((0,0))))
#print(sh.size)

# save to file
#fi.save('training.png')

['StopException', '_EVENTS', '_HANDLED_EXCEPTIONS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_bootstrap', '_bootstrap_inner', '_create_event_tap', '_delete', '_emitter', '_exc_info', '_handle', '_handler', '_initialized', '_mark_ready', '_reset_internal_locks', '_run', '_set_ident', '_set_tstate_lock', '_stop', '_wait_for_tstate_lock', 'daemon', 'getName', 'ident', 'isAlive', 'isDaemon', 'is_alive', 'join', 'name', 'run', 'running', 'setDaemon', 'setName', 'start', 'stop', 'wait']
Pressed at (295.1015625, 179.80078125)
Released at (295.1015625, 179.80078125)


In [29]:
a = ('r', (4,23))
print(a[1][1])

23
